# Installing MASE (again)

Run the block below to install MASE in the current Colab runtime

In [ ]:
git_token = "YOUR_GIT_TOKEN"
short_code = "YOUR_SHORT_CODE"

# Check the current python version (It should be using Python 3.10) and update pip to the latest version.
!python --version
!python -m pip install --user --upgrade pip

# Clone MASE from your branch (the branch must already exist)
!git clone -b lab1_{short_code} https://{git_token}@github.com/DeepWok/mase.git

# Install requirements
!python -m pip install -r ./mase/machop/requirements.txt

# Change working directory to machop
%cd ./mase/machop/

In [ ]:
!./ch --help

# General introduction

In this lab, you will learn how to use the software stack of MASE. There are in total 7 tasks you would need to finish, and 1 optional task.

# Turning you network to a graph

One specific feature of MASE is its capability to transform DL models to a computation graph using the [torch.fx](<https://pytorch.org/docs/stable/fx.html>) framework.


## Use the Transform functionality without CLI

This tutorial describes how to use the MASE transform functionality for a pre-trained model.

## Import related packages and machop

In [1]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

In [ ]:
!cd ~/dev/advanced-deep-learning-systems/mase/machop
!pwd


In [2]:
# figure out the correct path
machop_path = Path(".").resolve().parent.parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))

In [3]:
from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

INFO     Set logging level to info


## Set up the dataset

Here we create a `MaseDataModule` using the `jsc` dataset from lab1. Note the `MaseDataModule` also requires the name of the model you plan to use data module with. In this case it is `jsc-tiny`.

In [4]:
batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"


data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()


## Set up the model 

Here we use the previously trained `jsc-tiny` model in lab 1 as an example.

In [ ]:
# If you stored your model checkpoint on Google Drive, remember to mount the drive to the current runtime in order to access it
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/jsc-tiny_classification_jsc_2024-01-26/software/training_ckpts/best.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from /home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/jsc-tiny_classification_jsc_2024-01-26/software/training_ckpts/best.ckpt


# Get a dummy data in
With the dataset module and model information, we can grab an input generator.

In [6]:
# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)


## Generate a MaseGraph
We have two forms of passes: transform passes and analysis passes, both of them would require the model to be transferred into a MaseGraph to allow manipulation.

In [7]:
# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

## Running an Analysis pass
Analysis pass DOES NOT change the graph

The following analysis passes are essential to prepare the graph for other passes

In [8]:
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

We will first run a simple graph analysis to understand the structure of the model.

In [9]:
# report graph is an analysis pass that shows you the detailed information in the graph
from chop.passes.graph import report_graph_analysis_pass
_ = report_graph_analysis_pass(mg)

graph():
    %x : [num_users=1] = placeholder[target=x]
    %seq_blocks_0 : [num_users=1] = call_module[target=seq_blocks.0](args = (%x,), kwargs = {})
    %seq_blocks_1 : [num_users=1] = call_module[target=seq_blocks.1](args = (%seq_blocks_0,), kwargs = {})
    %seq_blocks_2 : [num_users=1] = call_module[target=seq_blocks.2](args = (%seq_blocks_1,), kwargs = {})
    %seq_blocks_3 : [num_users=1] = call_module[target=seq_blocks.3](args = (%seq_blocks_2,), kwargs = {})
    return seq_blocks_3
Network overview:
{'placeholder': 1, 'get_attr': 0, 'call_function': 0, 'call_method': 0, 'call_module': 4, 'output': 1}
Layer types:
[BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Linear(in_features=16, out_features=5, bias=True), ReLU(inplace=True)]


## Running another Analysis pass: Profile statistics

The pass `profile_statistics_analysis_pass` collects statistics of parameters and activations, and save them to node's metadata.

Here is a list of all the supported statistics. Refer to the `__init__` of statistic classes in `chop.passes.analysis.statistical_profiler.stat` to check the args each stat class takes.

This is a more complex analysis than the previous pass, and thus it would require you to pass in additional arguments for this pass.

### Example: the range of weights & input activations of nodes

Say we want to collect the tensor-wise min-max range of the 1st `torch.nn.Linear` nodes' weights & bias, and the channel-wise 97% quantile min-max of the 1st `torch.nn.Linear` nodes' input activations. We can do the following:

In [10]:

pass_args = {
    "by": "type",                                                            # collect statistics by node name
    "target_weight_nodes": ["linear"],                                       # collect weight statistics for linear layers
    "target_activation_nodes": ["relu"],                                     # collect activation statistics for relu layers
    "weight_statistics": {
        "variance_precise": {"device": "cpu", "dims": "all"},                # collect precise variance of the weight
    },
    "activation_statistics": {
        "range_quantile": {"device": "cpu", "dims": "all", "quantile": 0.97} # collect 97% quantile of the activation range
    },
    "input_generator": input_generator,                                      # the input generator for feeding data to the model
    "num_samples": 32,                                                       # feed 32 samples to the model
}

We can use the `report_node_meta_param_analysis_pass` to inspect the collected statistics.

In [11]:
mg, _ = profile_statistics_analysis_pass(mg, pass_args)
mg, _ = report_node_meta_param_analysis_pass(mg, {"which": ("software",)})

Profiling act statistics: 100%|██████████| 4/4 [00:00<00:00, 390.84it/s]
INFO     Inspecting graph [add_common_meta_param_analysis_pass]
INFO     
+--------------+--------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| Node name    | Fx Node op   | Mase type           | Mase op      | Software Param                                                                          |
+==============+==============+=====================+==============+=========================================================================================+
| x            | placeholder  | placeholder         | placeholder  | {'results': {'data_out_0': {'stat': {}}}}                                               |
+--------------+--------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| seq_blocks_0 | call_module  | module              | batc

## Running a Transform pass: Quantisation

As its name suggests, the transform pass would modify the `MaseGraph`.
Similar to the previous analysis pass example, we would need to first declare the configuration for the pass.

In [12]:
pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

We can then proceed to apply the transformation, in this case, we kept the original graph on purpose, so that we can print a `diff`.

In [13]:
from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph


ori_mg = MaseGraph(model=model)
ori_mg, _ = init_metadata_analysis_pass(ori_mg, None)
ori_mg, _ = add_common_metadata_analysis_pass(ori_mg, {"dummy_in": dummy_in})

mg, _ = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

INFO     Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       1 |         0 |           1 |
| Linear          | linear       |       1 |         1 |           0 |
| ReLU            | relu         |       2 |         0 |           2 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |


In [14]:
from pprint import pprint

pprint(ori_mg.modules['seq_blocks.2'])
pprint(ori_mg.modules['seq_blocks.2'].bias.data)
pprint(ori_mg.modules['seq_blocks.2'].weight.data)

pprint(mg.modules['seq_blocks.2'])
pprint(mg.modules['seq_blocks.2'].bias.data)
pprint(mg.modules['seq_blocks.2'].weight.data)

Linear(in_features=16, out_features=5, bias=True)
tensor([-0.1298,  0.2062,  0.4861, -0.2565, -0.1006])
tensor([[-0.2332,  0.2832, -0.1115, -0.0295, -0.0072,  0.1444,  0.0797,  0.3395,
          0.0553,  0.3004,  0.1721,  0.1641, -0.0056,  0.0201,  0.0698,  0.3321],
        [ 0.3941, -0.1131, -0.4779, -0.2654, -0.1232,  0.0928,  0.0162,  0.2843,
          0.0254,  0.1038,  0.3021, -0.0592, -0.0999,  0.0907, -0.4290, -0.0821],
        [ 0.0346,  0.1561, -0.1502, -0.5660, -0.1753,  0.0592,  0.1440,  0.0482,
          0.0399, -0.1276,  0.1169, -0.2150, -0.2085, -0.1641,  0.2179, -0.1539],
        [-0.0709, -0.0405,  0.0492,  0.1516,  0.0798, -0.0184,  0.2083, -0.1451,
          0.0881, -0.2306, -0.2248, -0.1836,  0.0951,  0.0654, -0.1739, -0.0221],
        [-0.1331,  0.0142,  0.2336,  0.2879,  0.3525,  0.4610, -0.2539, -0.0417,
          0.0423,  0.0954,  0.0374,  0.1446, -0.0673, -0.1901,  0.1603, -0.1296]])
LinearInteger(in_features=16, out_features=5, bias=True)
tensor([-0.1298,  0.206

In [15]:
from chop.passes.graph.utils import get_node_actual_target

for og_node, node in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):
    print("----------------------------------------------")
    print(f"original {og_node.name}: {get_node_actual_target(og_node)}")
    print(f"transformed {node.name}: {get_node_actual_target(node)}")

----------------------------------------------
original x: x
transformed x: x
----------------------------------------------
original seq_blocks_0: BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
transformed seq_blocks_0: BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------
original seq_blocks_1: ReLU(inplace=True)
transformed seq_blocks_1: ReLU(inplace=True)
----------------------------------------------
original seq_blocks_2: Linear(in_features=16, out_features=5, bias=True)
transformed seq_blocks_2: LinearInteger(in_features=16, out_features=5, bias=True)
----------------------------------------------
original seq_blocks_3: ReLU(inplace=True)
transformed seq_blocks_3: ReLU(inplace=True)
----------------------------------------------
original output: output
transformed output: output


In [16]:
from chop.passes.graph.utils import get_node_actual_target
from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph

batch_size = 8
model_name = "jsc-custom"
dataset_name = "jsc"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

LAB1_CUSTOM_PATH = "/home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/jsc-custom_classification_jsc_2024-01-27/software/training_ckpts/best.ckpt"
model_info = get_model_info(model_name)
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=LAB1_CUSTOM_PATH, load_type="pl", model=model)

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
    "relu": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # # weight
            # "weight_width": 8,
            # "weight_frac_width": 4,
            # # bias
            # "bias_width": 8,
            # "bias_frac_width": 4,
        }
    },
}

mg = MaseGraph(model=model)
mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})

mg_trans = MaseGraph(model=model)
mg_trans, _ = init_metadata_analysis_pass(mg_trans, None)
mg_trans, _ = add_common_metadata_analysis_pass(mg_trans, {"dummy_in": dummy_in})
mg_trans, _ = quantize_transform_pass(mg_trans, pass_args)
summarize_quantization_analysis_pass(mg, mg_trans, save_dir="quantize_summary")


for og_node, node in zip(mg.fx_graph.nodes, mg_trans.fx_graph.nodes):
    print("----------------------------------------------")
    print(f"original {og_node.name}: {get_node_actual_target(og_node)}")
    print(f"transformed {node.name}: {get_node_actual_target(node)}")

INFO     Loaded pytorch lightning checkpoint from /home/bkt123/dev/advanced-deep-learning-systems/mase/mase_output/jsc-custom_classification_jsc_2024-01-27/software/training_ckpts/best.ckpt
INFO     Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       4 |         0 |           4 |
| Dropout         | dropout      |       2 |         0 |           2 |
| Linear          | linear       |       3 |         3 |           0 |
| ReLU            | relu         |       4 |         4 |           0 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |


----------------------------------------------
original x: x
transformed x: x
----------------------------------------------
original seq_blocks_0: BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
transformed seq_blocks_0: BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------
original seq_blocks_1: ReLU(inplace=True)
transformed seq_blocks_1: ReLUInteger(inplace=True)
----------------------------------------------
original seq_blocks_2: Linear(in_features=16, out_features=32, bias=True)
transformed seq_blocks_2: LinearInteger(in_features=16, out_features=32, bias=True)
----------------------------------------------
original seq_blocks_3: BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
transformed seq_blocks_3: BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------
original seq_bloc

In [17]:
from chop.passes.graph.transforms.quantize.quantized_modules.linear import _LinearBase

i = 0
for module_name, module_trans_name in zip(mg.modules, mg_trans.modules):
    module = mg.modules[module_name]
    module_trans = mg_trans.modules[module_trans_name]

    if isinstance(module_trans, _LinearBase):
        print("Original weights: ", module.weight[:2])
        print("Quantised weights: ", module_trans.w_quantizer(module_trans.weight)[:2])
        i += 1
        
    if i == 3:
        break


Original weights:  tensor([[ 0.0255,  0.1090,  0.0359,  0.2054,  0.1749,  0.0137, -0.0309,  0.0114,
         -0.0637,  0.0512,  0.0183, -0.0041, -0.0681, -0.0890, -0.5769, -0.1234],
        [ 0.0558,  0.1819, -0.3261,  0.0708, -0.1033,  0.0756, -0.0145,  0.0285,
         -0.0013,  0.0439,  0.1017, -0.2077,  0.0847,  0.0008,  0.0621,  0.3468]],
       grad_fn=<SliceBackward0>)
Quantised weights:  tensor([[ 0.0000,  0.1250,  0.0625,  0.1875,  0.1875,  0.0000, -0.0000,  0.0000,
         -0.0625,  0.0625,  0.0000, -0.0000, -0.0625, -0.0625, -0.5625, -0.1250],
        [ 0.0625,  0.1875, -0.3125,  0.0625, -0.1250,  0.0625, -0.0000,  0.0000,
         -0.0000,  0.0625,  0.1250, -0.1875,  0.0625,  0.0000,  0.0625,  0.3750]],
       grad_fn=<SliceBackward0>)
Original weights:  tensor([[ 0.0825,  0.1261,  0.0914,  0.0158, -0.2281, -0.2152, -0.1013, -0.4228,
          0.1333, -0.1087,  0.0085, -0.0342, -0.3247,  0.0023,  0.0238, -0.1339,
         -0.0091, -0.3102,  0.1026, -0.0515,  0.1231, -0.027

In [18]:
from pprint import pprint

from chop.passes.graph.analysis.flops.flops_pass import analyse_flops_pass
from chop.passes.graph.utils import get_mase_op

analyse_flops_pass(mg)

Computations summary:
---------------------
| Node name     | Node type    |   Parameters |   Forward FLOPS |   Backward FLOPS |   Input buffer size |   Output buffer size |
|---------------|--------------|--------------|-----------------|------------------|---------------------|----------------------|
| x             | placeholder  |              |                 |                  |                     |                      |
| seq_blocks_0  | batch_norm1d |           32 |             512 |              512 |                 128 |                  128 |
| seq_blocks_1  | relu         |            0 |             128 |              128 |                 128 |                  128 |
| seq_blocks_2  | linear       |          512 |            4096 |             8192 |                 128 |                  256 |
| seq_blocks_3  | batch_norm1d |           64 |            1024 |             1024 |                 256 |                  256 |
| seq_blocks_4  | relu         |            0 

In [22]:
for node in mg.fx_graph.nodes:
    print(node.meta['mase'].parameters['common'])

{'mase_type': 'placeholder', 'mase_op': 'placeholder', 'args': {}, 'results': {'data_out_0': {'type': 'float', 'precision': [32], 'shape': [8, 16], 'torch_dtype': torch.float32, 'value': tensor([[-0.6870,  0.6464, -0.2211, -0.5086, -0.2880, -0.4539, -0.3727, -0.5476,
         -0.3727,  0.0828,  0.2902, -0.1872,  0.0218, -0.4286, -0.2975,  0.4106],
        [ 0.5310,  0.2459,  0.0614, -0.0658, -0.3883, -0.3701, -1.1023, -0.8840,
         -1.1023, -1.5075, -1.5469, -1.2582, -1.6790, -1.2448,  0.2650, -1.0364],
        [-0.2012,  0.3965,  0.9537,  0.9707,  0.1673, -0.3228, -1.0676, -1.1820,
         -1.0676, -1.1923, -1.0395, -1.1846, -1.2158, -1.3171,  1.0332,  0.6035],
        [ 1.0630, -0.3592, -1.1664, -0.8446, -1.0724, -0.6250, -0.2240, -0.1673,
         -0.2240,  0.4029,  0.6315,  0.7814,  0.0869,  0.0549, -1.2494, -0.8917],
        [-1.5865,  0.9318,  1.4369,  1.5644,  2.3059,  2.4386,  0.9330,  0.8453,
          0.9330,  0.8588,  0.7322,  0.5232,  1.0754,  0.9757,  1.4826,  1.8575]

In [20]:
from chop.passes.graph.analysis.flops.flops_pass import analyse_flops_pass

analyse_flops_pass(mg)

Computations summary:
---------------------
| Node name     | Node type    |   Parameters |   Forward FLOPS |   Backward FLOPS |   Input buffer size |   Output buffer size |
|---------------|--------------|--------------|-----------------|------------------|---------------------|----------------------|
| x             | placeholder  |              |                 |                  |                     |                      |
| seq_blocks_0  | batch_norm1d |           32 |             512 |              512 |                 128 |                  128 |
| seq_blocks_1  | relu         |            0 |             128 |              128 |                 128 |                  128 |
| seq_blocks_2  | linear       |          512 |            4096 |             8192 |                 128 |                  256 |
| seq_blocks_3  | batch_norm1d |           64 |            1024 |             1024 |                 256 |                  256 |
| seq_blocks_4  | relu         |            0 



# Exercises:

We have now seen how to:
1. Set up a dataset
2. Set up a model
3. Generate a `MaseGraph` from the model
4. Run Analysis and Transform passes on the `MaseGraph`

Now consider the following problems:

1. Explain the functionality of `report_graph_analysis_pass` and its printed jargons such as `placeholder`, `get_attr` ... You might find the doc of [torch.fx](https://pytorch.org/docs/stable/fx.html) useful.

2. What are the functionalities of `profile_statistics_analysis_pass` and `report_node_meta_param_analysis_pass` respectively?

### Answer 1
> Explain the functionality of `report_graph_analysis_pass` and its printed jargons such as `placeholder`, `get_attr` 

`report_graph_analysis_pass` steps through each node of the FX graph and counts how many of each node fit the following types:
- `placeholder`- method inputs
- `get_attr`, `call_function`, `call_method`, `call_module` - the operations within the node, whether or not the node is getting an attribute or calling one of a free function, a method or a module
- `output` - how many outputs of the graph

The pass also reports the layers in the graph along with their settings (example, `Linear(in_features=16, out_features=5, bias=True)`).

This pass seems like it would be useful for getting an understanding of how the model is structured.

### Answer 2
> What are the functionalities of `profile_statistics_analysis_pass` and `report_node_meta_param_analysis_pass` respectively?

`profile_statistics_analysis_pass` iterates through each node and checks whether it matches the arguments specified, and if it does then it computes statistics based on the dummy data passed in. Stats can be computed for weight nodes (where only `call_module` nodes are considered) and for activation nodes separately, and they can also be further filtered by a type of node (example `linear` or `relu`) or specifically by the name. Statistics such as the variance, the range, etc. can be computed.

`report_node_meta_param_analysis_pass` then iterates through the updated graph (processed by `profile_statistics_analysis_pass`) and outputs the stats computed.



## MASE OPs and MASE Types

MASE is designed to be a very high-level intermediate representation (IR), this is very different from the classic [LLVM IR](https://llvm.org/docs/LangRef.html) that you might be familiar with.

The following MASE Types are available:
(Note from Aaron: do we have a page somewhere that have summarized this?)


## A deeper dive into the quantisation transform

3. Explain why only 1 OP is changed after the `quantize_transform_pass` .

4. Write some code to traverse both `mg` and `ori_mg`, check and comment on the nodes in these two graphs. You might find the source code for the implementation of `summarize_quantization_analysis_pass` useful.

5. Perform the same quantisation flow to the bigger JSC network that you have trained in lab1. You must be aware that now the `pass_args` for your custom network might be different if you have used more than the `Linear` layer in your network.

6. Write code to show and verify that the weights of these layers are indeed quantised. You might need to go through the source code of the implementation of the quantisation pass and also the implementation of the [Quantized Layers](../../machop/chop/passes/transforms/quantize/quantized_modules/linear.py) .

### Answer 3
The `pass_args` specified that the quantisation should be done by `type` of `linear`, and there is only one linear layer in the model so only this was changed.

### Answer 4
The following basic code was written to traverse the nodes of `mg` and `ori_mg` and compare the nodes:
```python
for og_node, node in zip(ori_mg.fx_graph.nodes, mg.fx_graph.nodes):
    print("----------------------------------------------")
    print(f"original {og_node.name}: {get_node_actual_target(og_node)}")
    print(f"transformed {node.name}: {get_node_actual_target(node)}")
print("----------------------------------------------")
```

This code produced the following output:
```text
----------------------------------------------
original x: x
transformed x: x
----------------------------------------------
original seq_blocks_0: BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
transformed seq_blocks_0: BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
----------------------------------------------
original seq_blocks_1: ReLU(inplace=True)
transformed seq_blocks_1: ReLU(inplace=True)
----------------------------------------------
original seq_blocks_2: Linear(in_features=16, out_features=5, bias=True)
transformed seq_blocks_2: LinearInteger(in_features=16, out_features=5, bias=True)
----------------------------------------------
original seq_blocks_3: ReLU(inplace=True)
transformed seq_blocks_3: ReLU(inplace=True)
----------------------------------------------
original output: output
transformed output: output
----------------------------------------------
```

We can see that all the layers are identical except for `seq_blocks_2`, with the original being of type `Linear` and the new being `LinearInteger` after the quantization pass. This confirms the observations in the answer to Q3 above.


## The command line interface

The same flow can also be executed on the command line throw the `transform` action.

```bash
# make sure you have the same printout
pwd
# it should show
# your_dir/mase-tools/machop

# enter the following command
./ch transform --config configs/examples/jsc_toy_by_type.toml --task cls --cpu=0
```
7. Load your own pre-trained JSC network, and perform perform the quantisation using the command line interface.

## \[Optional] Write your own pass

Many examples of existing passes are in the [source code](../..//machop/chop/passes/__init__.py), the [test files](../../machop/test/passes) for these passes also contain useful information on helping you to understand how these passes are used.

Implement a pass to count the number of FLOPs (floating-point operations) and BitOPs (bit-wise operations).